In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import main as m
import regex as re

In [2]:
invest, assets, events, skills = m.get_urls_by_type()


Getting investigator URL's...
Getting asset URL's...
Getting event URL's...
Getting skill URL's...


In [3]:
##########################################Get soup request#########################################################
def get_soup(url):
    '''Child of get_skills_df
       Takes in a url for a card
       Returns html request result parsed using beautiful soup'''

    # create request and soup objects
    html = requests.get(url)

    soup = BeautifulSoup(html.content, 'html.parser')

    # locate urls on page and return
    return soup.find(id='list')

In [4]:
# 'title':[], # name + title
# 'faction':[],
# 'type':[],
# 'traits':[],                      
# 'willpower':[],
# 'intellect':[],
# 'combat':[],
# 'agility':[],                       
# 'health':[],
# 'sanity':[],                     
# 'ability':[],
# 'artist':[],
# 'expansion':[],
# 'deck_size':[],
# 'deckbuilding':[], # Options + Reqs

In [99]:
def get_stat_line(results):
    
    stat_line = results.find('div', class_='card-props').text.replace('\n', '').replace('\t', '')

    stat_list = [int(char) for char in str(stat_line) if char.isnumeric() == True]

    return stat_list[0], stat_list[1], stat_list[2], stat_list[3]


def get_stam_line(results):
    
    stat_line = results.find('div', class_='card-props').find_next().text.replace('\n', '').replace('\t', '')

    stat_list = [int(char) for char in str(stat_line) if char.isnumeric() == True]

    return stat_list[0], stat_list[1]


def get_deck_reqs(results):
    
    deck = results.find_all('div', class_=f'card-text border-{faction.lower()}')[1].text

    deck_split = re.split(':', deck)

    deck_size = re.search('[0-9]{2}', deck_split[1]).group()
    
    deck_options = re.search('^(.*?\.)', deck_split[2]).group()[1:-1].replace('(', '').replace(')', '').replace('  ', ' ')

    must_include = re.search('^(.*?\.)', deck_split[3]).group()[1:-1].replace('(', '').replace(')', '').replace('  ', ' ')
    
    return deck_size, deck_options, must_include

In [100]:
# # make df

#     # for each url get player card info from that page and add each element to skill_traits
#     for url in invest:

#         # make html request to arkham db and parse using BS
#         results = get_soup(url)

#         # get list of card elements card elements
#         trait_list = get_ability_traits(results)
        
#         print(f'Getting investigator card {title}...')

#         # itterate through card element titles and add each to a dictionary
#         for i, key in enumerate(investigator_traits):

#             investigator_traits[key].append(trait_list[i])

#     print("Making dataframe...")

#     # convert dictionary to dataframe
#     df_skill = pd.DataFrame(skill_traits)

#     return df_skill

In [101]:
# [title, test_icons, traits, willpower, intellect, combat, agility, health, sanity, faction, ability, tipe, flavor, artist, expansion]

In [119]:
# Investigator Traits

results = get_soup(invest[20])

title = results.find('a', class_='card-name card-tip').text.replace('\n', '').replace('\t', '')

faction = results.find('span', class_='card-faction').text.replace('\n', '').replace('\t', '')

tipe = results.find('span', class_='card-type').text.replace('\n', '').replace('\t', '')

traits = results.find('p', class_='card-traits').text.replace('\n', '').replace('\t', '')

willpower, intellect, combat, agility = get_stat_line(results)

health, sanity = get_stam_line(results)

ability = results.find('div', class_=f'card-text border-{faction.lower()}').text.replace('\n', '').replace('\t', '')

artist = results.find('div', class_='card-illustrator').text.replace('\n', '').replace('\t', '')

expansion = results.find('div', class_='card-pack').text.replace('\n', '').replace('\t', '').replace('.', '') 

flavor = results.find('div', class_='card-flavor small').text.replace('\n', '').replace('\t', '')

deck_size, deck_options, must_include = get_deck_reqs(results)

In [139]:
results.find('div', class_='card-flavor small').text.replace('\n', '').replace('\t', '')

''

In [86]:
results.find_all('div', class_=f'card-text border-{faction.lower()}')[1].text

'\nDeck Size: 35.Deckbuilding Options: Mystic cards () level 0-5, Guardian cards () level 0-2, Neutral cards level 0-5.Deckbuilding Requirements (do not count toward deck size): Twilight Blade, Dark Insight, Terrible Secret, 1 random basic weakness.Additional Setup: You begin each game with Dark Insight as an additional card in your opening hand.\n'

In [120]:
title

'Diana Stanley'

In [121]:
faction

'Mystic'

In [122]:
tipe

'Investigator'

In [123]:
traits

'Cultist. Silver Twilight.'

In [124]:
willpower

1

In [125]:
intellect

3

In [126]:
combat

3

In [127]:
agility

3

In [128]:
health

7

In [129]:
sanity

7

In [130]:
ability

'Diana Stanley gets +1  for each card beneath her. After a card you own cancels or ignores a card effect or game effect, if there are fewer than 5 cards beneath Diana Stanley: Place that card facedown beneath her. Draw 1 card and gain 1 resource. (Limit once per phase.) effect: +2. You may choose a card beneath Diana Stanley and add it to your hand.'

In [131]:
artist

'Magali Villeneuve'

In [132]:
expansion

'The Circle Undone #4'

In [133]:
deck_size

'35'

In [134]:
flavor

''

In [136]:
deck_size

'35'

In [137]:
deck_options

'Mystic cards level 0-5, Guardian cards level 0-2, Neutral cards level 0-5'

In [138]:
must_include 

'Twilight Blade, Dark Insight, Terrible Secret, 1 random basic weakness'